In [2]:
import pandas as pd
import numpy as np
import os

import psycopg2

In [3]:
file = open("db_login.txt", 'r')
logins = file.readlines()

for i, line in enumerate(logins):
    logins[i] = line.split('\n')[0]
    
file.close()

connect_params = {
     "host"      : logins[0],
     "database"  : logins[1],
     "user"      : logins[2],
     "password"  : logins[3]
}

In [4]:
def connect(conn_params):
    conn = None
    
    try:
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**conn_params)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
        
    print("Connection successful")
    
    return conn

In [5]:
conn = connect(connect_params)

Connecting to the PostgreSQL database...
Connection successful


In [6]:
def postgresql_to_dataframe(conn, select_query, column_headers):
    cursor = conn.cursor()
    
    try:
        cursor.execute(select_query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    
    tuples = cursor.fetchall()
    cursor.close()
    
    df = pd.DataFrame(tuples, columns=column_headers)
    return df

## Candidates Database

#### Congressional Candidates

In [6]:
# All Congressional Candidates
query1 = """
SELECT cycle, cid, first_last_party, party, dist_id_run_for, current_candidate, cycle_candidate, recip_code,
raised_from_pacs, raised_from_individuals, raised_total, raised_unitemized
FROM candidates
WHERE dist_id_run_for NOT LIKE 'PRES' AND dist_id_run_for NOT LIKE '__S_'
"""

In [7]:
col_names_1 = ['cycle', 'id', 'candidate_name', 'party', 'district', 'current_candidate', 'cycle_candidate',
               'recip_code', 'raised_from_pacs', 'raised_from_individuals', 'raised_total', 'raised_unitemized']

In [8]:
all_congr_candids = postgresql_to_dataframe(conn, query1, col_names_1)
all_congr_candids

,cycle,id,candidate_name,party,district,current_candidate,cycle_candidate,recip_code,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized
0,2000,N00005009,Richmond A Soluade Sr (R),R,MO01,,Y,RL,NaN,NaN,0,NaN
1,1996,N00004126,Mark Alan Behnke (R),R,MI07,,,RN,NaN,NaN,0,0.0
2,1996,N00000718,Jim Ford (R),R,NJ10,,,RN,NaN,NaN,0,0.0
3,1996,N00005551,Ernest J Istook (R),R,OK05,Y,Y,RW,130384.0,99600.0,399980,169996.0
4,1996,N00008523,Stephen Wayne Hofman (R),R,MI16,,Y,RL,NaN,5576.0,18050,12474.0
...,...,...,...,...,...,...,...,...,...,...,...,...
51143,2016,N00033316,Joaquin Castro (D),D,TX20,Y,Y,DI,211035.0,188278.0,431521,32208.0
51144,2016,N00025284,Raul M Grijalva (D),D,AZ03,Y,Y,DI,102018.0,41449.0,184909,41442.0
51145,2016,N00034130,Randal Wallace (R),R,SC07,,,RN,1000.0,4800.0,7575,1775.0
51146,2016,N00013846,Jeff Miller (R),R,FL01,Y,Y,RI,319963.0,403622.0,730935,7350.0


In [9]:
# Insert State Column
congr_states = [state[:2] for state in all_congr_candids['district']]
all_congr_candids.insert(4, 'state', congr_states)

# Extract Just District Number
districts = [state[-2:] for state in all_congr_candids['district']]
all_congr_candids['district'] = districts

# Remove Party Affliation from Name
all_congr_candids['candidate_name'] = [name[:-4] for name in all_congr_candids['candidate_name']]

In [10]:
all_congr_candids.head()

,cycle,id,candidate_name,party,state,district,current_candidate,cycle_candidate,recip_code,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized
0,2000,N00005009,Richmond A Soluade Sr,R,MO,01,,Y,RL,NaN,NaN,0,NaN
1,1996,N00004126,Mark Alan Behnke,R,MI,07,,,RN,NaN,NaN,0,0.0
2,1996,N00000718,Jim Ford,R,NJ,10,,,RN,NaN,NaN,0,0.0
3,1996,N00005551,Ernest J Istook,R,OK,05,Y,Y,RW,130384.0,99600.0,399980,169996.0
4,1996,N00008523,Stephen Wayne Hofman,R,MI,16,,Y,RL,NaN,5576.0,18050,12474.0


#### Senate Candidates

In [11]:
# All Senate Candidates
query2 = """
SELECT cycle, cid, first_last_party, party, dist_id_run_for, current_candidate, cycle_candidate, recip_code,
raised_from_pacs, raised_from_individuals, raised_total, raised_unitemized
FROM candidates
WHERE dist_id_run_for NOT LIKE 'PRES' AND dist_id_run_for LIKE '__S_'
"""

In [12]:
col_names_2 = ['cycle', 'id', 'candidate_name', 'party', 'district', 'current_candidate', 'cycle_candidate',
               'recip_code', 'raised_from_pacs', 'raised_from_individuals', 'raised_total', 'raised_unitemized']

In [13]:
all_sen_candids = postgresql_to_dataframe(conn, query2, col_names_2)
all_sen_candids

,cycle,id,candidate_name,party,district,current_candidate,cycle_candidate,recip_code,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized
0,1996,N00004451,Stephen Bonsal Young (R),R,MNS1,,Y,RL,492.0,97408.0,264254,166354.0
1,1996,N00001670,Raymond J Clatworthy (R),R,DES2,Y,Y,RL,202815.0,618935.0,1636414,814664.0
2,1996,N00000525,Gavin Terence Mills (I),I,VTS1,,,3L,NaN,266.0,266,0.0
3,1996,N00006284,James A McClure (R),R,IDS1,,,RN,NaN,NaN,0,0.0
4,1996,N00000421,Dick Swett (D),D,NHS2,Y,Y,DL,528713.0,1161345.0,3020794,1330736.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10728,1996,N00005453,Mark Brown (D),D,ARS1,,,DL,NaN,720.0,720,NaN
10729,2016,N00037568,Sean Guthrie (I),I,FLS2,Y,Y,3O,NaN,101.0,101,NaN
10730,2016,N00013820,Chris Van Hollen (D),D,MDS2,,,DO,209184.0,4032126.0,4643648,402338.0
10731,2016,N00007836,Maria Cantwell (D),D,WAS1,,,DI,-1750.0,122060.0,302913,182603.0


In [14]:
# Insert State Column
sen_states = [state[:2] for state in all_sen_candids['district']]
all_sen_candids.insert(4, 'state', sen_states)

# Replace all District Numbers with Null Values
all_sen_candids['district'] = [np.nan for district in all_sen_candids['district']]

# Remove Party Affliation from Name
all_sen_candids['candidate_name'] = [name[:-4] for name in all_sen_candids['candidate_name']]

In [15]:
all_sen_candids.head()

,cycle,id,candidate_name,party,state,district,current_candidate,cycle_candidate,recip_code,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized
0,1996,N00004451,Stephen Bonsal Young,R,MN,NaN,,Y,RL,492.0,97408.0,264254,166354.0
1,1996,N00001670,Raymond J Clatworthy,R,DE,NaN,Y,Y,RL,202815.0,618935.0,1636414,814664.0
2,1996,N00000525,Gavin Terence Mills,I,VT,NaN,,,3L,NaN,266.0,266,0.0
3,1996,N00006284,James A McClure,R,ID,NaN,,,RN,NaN,NaN,0,0.0
4,1996,N00000421,Dick Swett,D,NH,NaN,Y,Y,DL,528713.0,1161345.0,3020794,1330736.0


In [16]:
# Append both House and Senate Candidate Info
all_candidates = all_congr_candids.append(all_sen_candids)
all_candidates

,cycle,id,candidate_name,party,state,district,current_candidate,cycle_candidate,recip_code,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized
0,2000,N00005009,Richmond A Soluade Sr,R,MO,01,,Y,RL,NaN,NaN,0,NaN
1,1996,N00004126,Mark Alan Behnke,R,MI,07,,,RN,NaN,NaN,0,0.0
2,1996,N00000718,Jim Ford,R,NJ,10,,,RN,NaN,NaN,0,0.0
3,1996,N00005551,Ernest J Istook,R,OK,05,Y,Y,RW,130384.0,99600.0,399980,169996.0
4,1996,N00008523,Stephen Wayne Hofman,R,MI,16,,Y,RL,NaN,5576.0,18050,12474.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10728,1996,N00005453,Mark Brown,D,AR,NaN,,,DL,NaN,720.0,720,NaN
10729,2016,N00037568,Sean Guthrie,I,FL,NaN,Y,Y,3O,NaN,101.0,101,NaN
10730,2016,N00013820,Chris Van Hollen,D,MD,NaN,,,DO,209184.0,4032126.0,4643648,402338.0
10731,2016,N00007836,Maria Cantwell,D,WA,NaN,,,DI,-1750.0,122060.0,302913,182603.0


In [17]:
# Exclude any candidates from U.S. Territories
us_territories = ['AS', 'GU', 'MH', 'MP', 'PR', 'Pr', 'VI']
boolean_filter = ~all_candidates.state.isin(us_territories)
all_CONUS_candidates = all_candidates[boolean_filter]
all_CONUS_candidates

,cycle,id,candidate_name,party,state,district,current_candidate,cycle_candidate,recip_code,raised_from_pacs,raised_from_individuals,raised_total,raised_unitemized
0,2000,N00005009,Richmond A Soluade Sr,R,MO,01,,Y,RL,NaN,NaN,0,NaN
1,1996,N00004126,Mark Alan Behnke,R,MI,07,,,RN,NaN,NaN,0,0.0
2,1996,N00000718,Jim Ford,R,NJ,10,,,RN,NaN,NaN,0,0.0
3,1996,N00005551,Ernest J Istook,R,OK,05,Y,Y,RW,130384.0,99600.0,399980,169996.0
4,1996,N00008523,Stephen Wayne Hofman,R,MI,16,,Y,RL,NaN,5576.0,18050,12474.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10728,1996,N00005453,Mark Brown,D,AR,NaN,,,DL,NaN,720.0,720,NaN
10729,2016,N00037568,Sean Guthrie,I,FL,NaN,Y,Y,3O,NaN,101.0,101,NaN
10730,2016,N00013820,Chris Van Hollen,D,MD,NaN,,,DO,209184.0,4032126.0,4643648,402338.0
10731,2016,N00007836,Maria Cantwell,D,WA,NaN,,,DI,-1750.0,122060.0,302913,182603.0


In [18]:
all_CONUS_candidates.to_csv('../data/all_CONUS_candidates.csv', index=False)

---
# Individual Donations Database

In [19]:
cycle = 1990

#### Senate Campaigns

In [20]:
# All Senate Individual Donations
query3 = """
SELECT DISTINCT(i.fec_trans_id), i.cycle, i.date_donated, i.recipient_id, i.contributor_id, i.contributor_name,
i.org_name, i.ult_org, i.amount, i.city, i.state, i.recip_code, i.type, i.gender, i.occupation, i.employer
FROM individual_contributions AS i
INNER JOIN candidates AS c on i.recipient_id = c.cid
WHERE i.cycle="""+str(cycle)+""" AND i.recipient_id LIKE 'N%' AND
c.dist_id_run_for NOT LIKE 'PRES' AND c.dist_id_run_for LIKE '__S_'
"""

In [21]:
col_names_3 = ['transaction_id', 'cycle', 'date', 'recipient_id', 'contributor_id', 'contributor_name',
               'org_name', 'ult_org', 'amount', 'city', 'state', 'recip_code', 'type',
               'gender', 'occupation', 'employer']

In [22]:
df1 = postgresql_to_dataframe(conn, query3, col_names_3)
df1.head(5)

,transaction_id,cycle,date,recipient_id,contributor_id,contributor_name,org_name,ult_org,amount,city,state,recip_code,type,gender,occupation,employer
0,0000103,1990,1989-01-08,N00003072,a0000000103,"WILLIAMSON, J H",Independent Oil Dealer,,500.0,ANNISTON,AL,DW,15,I,None,None
1,0000114,1990,1989-01-07,N00003072,a00000886641,"BUTTRAM, H DEAN JR",Attorney,,1000.0,CENTRE,AL,DW,15,M,None,None
2,0000132,1990,1989-01-03,N00003072,a0000000132,"TREDAWAY, FLOYD P",Retired,,500.0,JACKSONVILLE,AL,DW,15,M,None,None
3,0000133,1990,1989-01-08,N00003072,a00000001331,"KLIMASEWSKI, LINDA",Ft McClellan School,,250.0,JACKSONVILLE,AL,DW,15,F,None,None
4,0000135,1990,1989-01-08,N00003072,a00000001331,"KLIMASEWSKI, LINDA",Ft McClellan School,,250.0,JACKSONVILLE,AL,DW,15,F,None,None


In [23]:
df1.to_csv('../data/individual_contributions/senate/sen_'+str(cycle)+'_individ_donations.csv', index=False)

#### Congressional Campaigns

In [24]:
# All Congressional Individual Donations
query4 = """
SELECT DISTINCT(i.fec_trans_id), i.cycle, i.date_donated, i.recipient_id, i.contributor_id, i.contributor_name, i.org_name,
i.ult_org, i.amount, i.city, i.state, i.recip_code, i.type, i.gender, i.occupation, i.employer
FROM individual_contributions AS i
INNER JOIN candidates AS c on i.recipient_id = c.cid
WHERE i.cycle="""+str(cycle)+""" AND i.recipient_id LIKE 'N%' AND
c.dist_id_run_for NOT LIKE 'PRES' AND c.dist_id_run_for NOT LIKE '__S_'
"""

In [25]:
df2 = postgresql_to_dataframe(conn, query4, col_names_3)
df2.tail(5)

,transaction_id,cycle,date,recipient_id,contributor_id,contributor_name,org_name,ult_org,amount,city,state,recip_code,type,gender,occupation,employer
168083,0777711,1990,1989-06-23,N00005998,a0001130787,"NEAL, LOYD",Hrh Insurance,,1000.0,CORPUS CHRISTI,TX,DW,15,M,None,None
168084,0777712,1990,1989-06-20,N00005998,a0000933875,"REYES, JOSEPH",Reyes & Assoc,,500.0,POTOMAC,MD,DW,15,M,None,None
168085,0777713,1990,1989-06-28,N00005998,a0000961572,"STORM, JAMES C",Oil Drilling,,1000.0,CORPUS CHRISTI,TX,DW,15,M,None,None
168086,0777714,1990,1989-06-28,N00005998,a0001156677,"STORM, RALPH",Drilling,,300.0,CORPUS CHRISTI,TX,DW,15,M,None,None
168087,0777726,1990,1989-02-09,N00005347,a0000677407,"LAMBERT, LAURENCE L",Engineer,,200.0,KENNER,LA,RN,15,M,None,None


In [26]:
df2.to_csv('../data/individual_contributions/house/congr_'+str(cycle)+'_individ_donations.csv', index=False)

---
# PAC Donations

In [99]:
cycle = 2016

In [100]:
# All PAC Donations
query5 = """
SELECT p.cycle, p.date_donated, p.pac_id, r.pac_short, p.amount, p.cid, p.real_code, p.type, r.recip_code, p.di
FROM pacs AS p
INNER JOIN pac_records AS r ON p.pac_id = r.committee_id
WHERE p.cycle = """+str(cycle)

In [101]:
col_names_5 = ['cycle', 'date_donated', 'pac_id', 'pac_name', 'amount', 'candidate_id', 'industry_code',
               'type', 'recip_code', 'direct']

In [102]:
df3 = postgresql_to_dataframe(conn, query5, col_names_5)
df3

,cycle,date_donated,pac_id,pac_name,amount,candidate_id,industry_code,type,recip_code,direct
0,2016,2015-05-15,C00097568,Raytheon Co,1000.0,N00030780,T1700,24K,PB,D
1,2016,2015-07-09,C00007880,Credit Union National Assn,1000.0,N00031104,F1300,24K,PB,D
2,2016,2015-06-02,C00002972,National Rural Electric Cooperative Assn,2500.0,N00029077,E1610,24K,PB,D
3,2016,2015-02-04,C00235655,Bluegrass Cmte,5000.0,N00002221,J2200,24K,PI,D
4,2016,2015-05-06,C00068528,Pacific Mutual Life,2500.0,N00024922,F3300,24K,PB,D
...,...,...,...,...,...,...,...,...,...,...
85864,2016,2015-06-22,C00544957,Emergency Medicine Physicians Inc,250.0,N00024842,H1130,24K,PB,D
85865,2016,2015-06-26,C00027342,Intl Brotherhood of Electrical Workers,-2500.0,N00000153,LC150,24K,PL,D
85866,2016,2015-02-03,C00131607,Florida Citrus Mutual,2500.0,N00030612,A1400,24K,PB,D
85867,2016,2015-07-14,C00139071,American Institute of Architects,5000.0,N00002408,B4200,24K,PB,D


In [103]:
df3.to_csv('../data/pacs/'+str(cycle)+'_pac_donations.csv', index=False)

In [105]:
conn.close()